In [5]:
from ftplib import FTP
import ftplib
import time
import tarfile
import os
import datetime
import sys
from optparse import OptionParser
import optparse

def ftpconnect(host, username, password):
    ftp = FTP()
    ftp.connect(host, 21)
    ftp.login(username, password)
    return ftp

#从ftp下载文件
def downloadfile(ftp, remotepath, localpath):
    bufsize = 1024
    fp = open(localpath, 'wb')
    ftp.retrbinary('RETR ' + remotepath, fp.write, bufsize)
    ftp.set_debuglevel(0)
    fp.close()

    
def main():
    parser = OptionParser(usage = "usage: %prog [option] arg1")
    parser.add_option("-s","--startdate",
                     action = "store",type= "string",dest = 'StartDate',default = "Latest",
                     help = "Type the StartDate, otherwise it will be the Latest Date of What You Have in Your Computer",)
    parser.add_option("-e","--enddate",
                     action = "store",type= "string",dest = 'EndDate', default = "Today",
                     help = "Type the EndDate, otherwise it will be Today",)
    parser.add_option("-f","--from",
                     action = "store",type = "string",dest = "UserID",
                     help = "FtpAddress:UserID:password",)
    parser.add_option("-t","--to",
                     action = "store",type = "string",dest = "SaveAddress",default = "C:/Users/kinsly/Work",
                     help = "Type the Address you want to save",)
    #args = ["-s","20170504"]
    (option,args) = parser.parse_args()
    #parser.print_help()

    today =datetime.date.today()- datetime.timedelta(1)   #获取今天日期
    today = str(today).replace('-','')
    print(option.SaveAddress)
    print(option.StartDate)
    print(option.EndDate)
    if(option.StartDate == 'Latest') or (option.EndDate == 'Today'):
        ZZAddress = option.SaveAddress
        print(ZZAddress)
        os.chdir(ZZAddress) ## Set working directory
        DirectoryList = os.listdir()
        DataDate = []
        ExchangeList = ['DL','SQ','ZZ','ZJ','SH','SZ']
        for item in DirectoryList:
            if item[0:2] in ExchangeList:
                DataDate.append(item[2:10])
        LatestDate = max(DataDate) # Find the Latest Date
        X1 = LatestDate
        X2 = today
    elif(option.StartDate != 'Latest') and (option.EndDate != 'Today'):
        X1 = option.StartDate
        X2 = option.EndDate
    else:
        print('{}'.format('You Enter the Wrong Number. Do You Really Understand 英国历史?'))
        sys.exit()
    X1 = datetime.datetime.strptime(X1,'%Y%m%d').date() ## 输入两个日期，判断两个日期大小
    X2 = datetime.datetime.strptime(X2,'%Y%m%d').date()
    
    if X1 > X2:
        DateFar = X2
        DateNear = X1
    else:
        DateFar = X1
        DateNear = X2  
    DateList = []
    length = (DateNear - DateFar).days
    for i in range(length+1):
        DateList.append(str(DateFar + datetime.timedelta(i)).replace('-','')) ## 建立一个日期列表
    
    
    #address = ['DL','SQ','ZZ','ZJ','SH','SZ']
    
    DownloadAddress = []
    x = option.SaveAddress
    ftp = ftpconnect(xxxxxx) # 输入正确的网址
    BlankList = []
    for DateItems in DateList:
        #address = ['ZJ'] ## 抓取中金所
        address = ['DL','SQ','ZZ','ZJ','SH','SZ']
        NewAddress = []
        DownloadAddress = []
        for items in address:
            NewAddress.append("/" + items + "/" + items + DateItems + ".rar") ## ftp索引路径
            DownloadAddress.append(x + '/' + items + DateItems + '.rar') ## 电脑本地路径
        
        AddressDict = dict(zip(NewAddress,DownloadAddress))
        for k, v in zip(NewAddress,DownloadAddress):
            print('Save    ' + v)
            try:
                downloadfile(ftp, k, v)
            except ftplib.all_errors:
                print(DateItems + ' is not a working day')
                BlankList.append(v)
    ftp.quit()
    for item in BlankList:
        print('Delete  ' + item)
        os.remove(item)
    print('Congrats! We are done!:)\n你太优秀啦！！！')
    
if __name__ == '__main__':
    main()
